In [1]:
import pygame
import tensorflow as tf
from tensorflow.contrib.rnn import RNNCell
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from func.jupyter_tensorboard import show_graph

from func.midi_to_statematrix import *
from func.data import *
import func.multi_training



import os
#import cPickle as pickle
import pickle
import signal
import numpy as np
import random

path = os.getcwd()



In [2]:
def play_music(music_file):
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print("Music file %s loaded!" % music_file)
    except pygame.error:
        print("File %s not found! (%s)" % (music_file, pygame.get_error()))
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        
        clock.tick(30)
        pygame.quit()

freq = 44100    # audio CD quality
bitsize = -16   # unsigned 16 bit
channels = 2    # 1 is mono, 2 is stereo
buffer = 1024    # number of samples
pygame.mixer.init(freq, bitsize, channels, buffer)
# optional volume 0 to 1.0
pygame.mixer.music.set_volume(0.8)



In [ ]:
play_music(path + '/music_test/' + 'beethoven_opus10_1.mid')

In [3]:
pcs = func.multi_training.loadPieces(path  + '/music_test')

Loaded alb_esp2
Loaded alb_esp5
Loaded appass_2
Loaded appass_3
Loaded bach_846
Loaded bach_847
Loaded bach_850
Loaded beethoven_hammerklavier_1
Loaded beethoven_les_adieux_1
Loaded beethoven_les_adieux_2
Loaded beethoven_opus10_2
Loaded beethoven_opus10_3
Loaded beethoven_opus22_1
Loaded beethoven_opus22_4
Loaded beethoven_opus90_2


In [4]:
batch_width = 10 # number of sequences in a batch
batch_len = 16*8 # length of each sequence
division_len = 16 # interval between possible start locations

def loadPieces(dirpath):

    pieces = {}

    for fname in os.listdir(dirpath):
        if fname[-4:] not in ('.mid','.MID'):
            continue

        name = fname[:-4]

        outMatrix = midiToNoteStateMatrix(os.path.join(dirpath, fname))
        if len(outMatrix) < batch_len:
            continue

        pieces[name] = outMatrix
        print("Loaded {}".format(name))

    return pieces

def getPieceSegment(pieces):
    pcs=pieces.values()
    piece_output = random.choice(list(pcs))
    start = random.randrange(0,len(piece_output)-batch_len,division_len)
    
    # print "Range is {} {} {} -> {}".format(0,len(piece_output)-batch_len,division_len, start)

    seg_out = piece_output[start:start+batch_len]
    seg_in = noteStateMatrixToInputForm(seg_out)

    return seg_in, seg_out

def getPieceBatch(pieces):
    i,o = zip(*[getPieceSegment(pieces) for _ in range(batch_width)])
    return numpy.array(i), numpy.array(o)

In [5]:
def trainPiece(model,pieces,epochs,start=0):
    stopflag = [False]
    def signal_handler(signame, sf):
        stopflag[0] = True
    old_handler = signal.signal(signal.SIGINT, signal_handler)
    for i in range(start,start+epochs):
        if stopflag[0]:
            break
        error = model.update_fun(*getPieceBatch(pieces))
        if i % 100 == 0:
            print("epoch {}, error={}".format(i,error))
        if i % 500 == 0 or (i % 100 == 0 and i < 1000):
            xIpt, xOpt = map(numpy.array, getPieceSegment(pieces))
            noteStateMatrixToMidi(numpy.concatenate((numpy.expand_dims(xOpt[0], 0), model.predict_fun(batch_len, 1, xIpt[0])), axis=0),'output/sample{}'.format(i))
            pickle.dump(model.learned_config,open('output/params{}.p'.format(i), 'wb'))
    signal.signal(signal.SIGINT, old_handler)

In [33]:
def reshape_time(x):
    
    input_slice = x[:,0:-1]
    n_batch, n_time, n_note, n_ipn = input_slice.shape
        
    # time_inputs is a matrix (time, batch/note, input_per_note)
    time_inputs = input_slice.transpose((1,0,2,3)).reshape((n_time,n_batch*n_note,n_ipn))
    return time_inputs,n_batch,n_time,n_note,n_ipn

In [29]:
song={}
song['beethoven_hammerklavier_1']=pcs['beethoven_hammerklavier_1']

In [40]:
def Model(t_layer_sizes,p_layer_sizes,xs,ys,n_time,n_batch,n_note,n_ipn):
    
    
    #n_batch, n_time, n_note, n_ipn = tf.shape(input_slice)[0],tf.shape(input_slice)[1],tf.shape(input_slice)[2],tf.shape(input_slice)[3]
    
    
    
    
    t_input_size = 80
    lstm_time=[]
    for i in t_layer_sizes:
        lstm_time.append(tf.contrib.rnn.LSTMCell(i))

    time_model=tf.contrib.rnn.MultiRNNCell(lstm_time)        
    init_state_time=time_model.zero_state(tf.shape(ys)[0],tf.float32)
    
    
    
    #output, self.final_state = tf.nn.dynamic_rnn(...)


    
    
    
    with tf.variable_scope('lstm1'):
        #tf.get_variable
        outputs_time,final_state_time=tf.nn.dynamic_rnn(time_model, xs, dtype = tf.float32)
        for one_lstm_cell in lstm_time:
            one_kernel, one_bias = one_lstm_cell.variables
            # I think TensorBoard handles summaries with the same name fine.
            tf.summary.histogram("Kernel-time", one_kernel)
            tf.summary.histogram("Bias-time", one_bias)    
    p_input_size = t_layer_sizes[-1] + 2

    # Transpose to be (note, batch/time, hidden_states)
    n_hidden = outputs_time.get_shape().as_list()[2]
    print(n_hidden)
    time_final = tf.reshape(tf.transpose(tf.reshape(outputs_time,(n_time,n_batch,n_note,n_hidden)),(2,1,0,3)),(n_note,n_batch*n_time,n_hidden))
    print(xs.shape) 
    print(time_final.shape) 
    
    # note_choices_inputs represents the last chosen note. Starts with [0,0], doesn't include last note.
    # In (note, batch/time, 2) format
    # Shape of start is thus (1, N, 2), concatenated with all but last element of output_mat transformed to (x, N, 2)
    ##start_note_values = T.alloc(np.array(0,dtype=np.int8), 1, tf.shape(time_final)[1], 2 )
    ##correct_choices = self.output_mat[:,1:,0:-1,:].transpose((2,0,1,3)).reshape((n_note-1,n_batch*n_time,2))
    ##note_choices_inputs = T.concatenate([start_note_values, correct_choices], axis=0)
        
    # Together, this and the output from the last LSTM goes to the new LSTM, but rotated, so that the batches in
    # one direction are the steps in the other, and vice versa.
    ##note_inputs = T.concatenate( [time_final, note_choices_inputs], axis=2 )
    ##num_timebatch = note_inputs.shape[1]
        
    
    
    
    
    lstm_pitch=[]

    for i in p_layer_sizes:
        lstm_pitch.append(tf.contrib.rnn.LSTMCell(i))
    lstm_pitch.append(tf.contrib.rnn.LSTMCell(2))


    pitch_model=tf.contrib.rnn.MultiRNNCell(lstm_pitch)
    print(tf.shape(time_final))
    init_state_pitch=pitch_model.zero_state(tf.shape(time_final)[0],tf.float32)
    with tf.variable_scope('lstm2'):
        outputs_pitch,final_state_pitch=tf.nn.dynamic_rnn(pitch_model,time_final,dtype = tf.float32)
        for one_lstm_cell in lstm_pitch:
            one_kernel, one_bias = one_lstm_cell.variables
            # I think TensorBoard handles summaries with the same name fine.
            
            tf.summary.histogram("Kernel-pitch", one_kernel)
            tf.summary.histogram("Bias-pitch", one_bias)    
        #variables_names =[v.name for v in tf.trainable_variables()]
        #values = sess.run(variables_names)
                
        #for k,v in zip(variables_names, values):
            #tf.summary.histogram('parameters/{}'.format(k),v)
                    
    return outputs_pitch


def cross_entropy(output, input_y):
    with tf.name_scope('cross_entropy'):
        
        ce = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=input_y, logits=output))

    return ce


def train_step(loss, learning_rate=1e-3):
    with tf.name_scope('train_step'):
        step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    return step


def evaluate(output, input_y):
    with tf.name_scope('evaluate'):
        pred = tf.argmax(output, axis=1)
        error_num = tf.count_nonzero(pred - tf.cast(input_y, tf.int64), name='error_num')
        
    return error_num

def training(song,t_layer_sizes,p_layer_sizes, pre_trained_model=None):
    
    tf.reset_default_graph()
    # define the variables and parameter needed during training
    with tf.name_scope('inputs'):
        xs = tf.placeholder(tf.float32, [None,None, t_input_size])
        ys = tf.placeholder(tf.float32, [None,None, 2])
        nbatch = tf.placeholder(tf.int32)
        ntime = tf.placeholder(tf.int32)
        nnote = tf.placeholder(tf.int32)
        nipn = tf.placeholder(tf.int32)
        
    output= Model(t_layer_sizes,p_layer_sizes,xs,ys,nbatch,ntime,nnote,nipn)
    
    print(output.shape)
    loss=cross_entropy(output, ys)
    
    iters = int(np.array(list(song.values())[0]).shape[0] / batch_len)
    print('number of batches for training: {}'.format(iters))

    step = train_step(loss)
    eve = evaluate(output, ys)

    iter_total = 0
    best_acc = 0
    cur_model_name = 'amadeus'

    epoch=1
    
    with tf.Session() as sess:
        
        merge = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter("log/{}".format(cur_model_name), sess.graph)
        saver = tf.train.Saver()
        
        sess.run(tf.global_variables_initializer())
        #train_writer = tf.summary.FileWriter("log/", graph=tf.get_default_graph())

        # try to restore the pre_trained
        if pre_trained_model is not None:
            try:
                print("Load the model from: {}".format(pre_trained_model))
                saver.restore(sess, 'model_save/{}'.format(pre_trained_model))
            except Exception:
                print("Load model Failed!")
                pass

        for epc in range(epoch):
            print("epoch {} ".format(epc + 1))

            for itr in range(iters):
                
                
                
                
                training_batch_x,training_batch_y= map(numpy.array, getPieceBatch(song))
                xtrain,n_batch,n_time,n_note,n_ipn=reshape_time(training_batch_x)
                
                ytrain,_,_,_,_=reshape_time(training_batch_y)
                print(xs.shape)
                print(ys.shape)
                print(xtrain.shape)
                print(ytrain.shape)
                print(n_batch,ntime,n_note,n_ipn)
                _, cur_loss = sess.run([step, loss], feed_dict={xs: xtrain, ys:ytrain, nbatch : n_batch,ntime : n_time, nnote : n_note, nipn : n_ipn})
                #out=sess.run(output, feed_dict={xs: training_batch_x, ys: training_batch_y})
                
                #variables_names =[v.name for v in tf.trainable_variables()]
                #print(variables_names)
                    
                merge_result=sess.run(merge, feed_dict={xs: xtrain, ys: ytrain})
                
                
                
                
                saver.save(sess,'model_save/{}'.format(cur_model_name))
                
                
                
                train_writer.add_summary(merge_result,itr)
               
                
                print(cur_loss)
                if itr==iters-1:
                    out=sess.run(output, feed_dict={xs: training_batch_x, ys: training_batch_y})
                    
                    
                    
                    u=noteStateMatrixToMidi(out, name="output")
                    
                    return out
                    #print(u)
                    
                    
                



In [41]:
t_input_size=80

In [42]:
a=training(song,[300,300],[100,50], pre_trained_model=None)

300
(?, ?, 80)
(?, ?, 300)
Tensor("Shape_1:0", shape=(3,), dtype=int32)
(?, ?, 2)
number of batches for training: 33


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x112f33f28>
Traceback (most recent call last):
  File "/Users/nathantoubiana/anaconda/lib/python3.5/weakref.py", line 356, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt


epoch 1 
(?, ?, 80)
(?, ?, 2)
(127, 780, 80)
(127, 780, 2)
10 Tensor("inputs/Placeholder_3:0", dtype=int32) 78 80


InvalidArgumentError: Incompatible shapes: [78,1270,2] vs. [127,780,2]
	 [[Node: train_step/gradients/cross_entropy/logistic_loss/mul_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](train_step/gradients/cross_entropy/logistic_loss/mul_grad/Shape, train_step/gradients/cross_entropy/logistic_loss/mul_grad/Shape_1)]]

Caused by op 'train_step/gradients/cross_entropy/logistic_loss/mul_grad/BroadcastGradientArgs', defined at:
  File "/Users/nathantoubiana/anaconda/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/nathantoubiana/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/nathantoubiana/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/nathantoubiana/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
   

In [ ]:
[78,1270,2] vs. [127,780,2]